In [1]:
# Load dependencies
import pandas as pd
from sqlalchemy import create_engine

## Extract the cleaned data into DataFrames

In [2]:
# Read in the cleaned speaker file, convert to a pandas dataframe, and display
speaker_file = "Resource/office_lines_cleaned.csv"
speaker_df = pd.read_csv(speaker_file)
speaker_df.head()

,Season,episode,Title,speaker,line
0,1,1,Pilot,Michael,All right Jim. Your quarterlies look very good...
1,1,1,Pilot,Jim,"Oh, I told you. I couldn't close it. So..."
2,1,1,Pilot,Michael,So you've come to the master for guidance? Is ...
3,1,1,Pilot,Jim,"Actually, you called me in here, but yeah."
4,1,1,Pilot,Michael,"All right. Well, let me show you how it's done."


In [3]:
# Read in the cleaned imdb ratings file, convert to a pandas dataframe, and display
imdb_file = "Resource/cleaned_imdb_dataset.csv"
imdb_df = pd.read_csv(imdb_file)
imdb_df.head()

,Season,Title,AirDate,Rating,Num_Votes,Description,DirectedBy,WrittenBy
0,1,Pilot,3/24/2005,7.5,4349,The premiere episode introduces the boss and s...,Ken Kwapis,Ricky Gervais |Stephen Merchant and Greg Daniels
1,1,Diversity Day,3/29/2005,8.3,4213,Michael's off color remark puts a sensitivity ...,Ken Kwapis,B. J. Novak
2,1,Health Care,4/5/2005,7.8,3536,Michael leaves Dwight in charge of picking the...,Ken Whittingham,Paul Lieberstein
3,1,The Alliance,4/12/2005,8.1,3428,"Just for a laugh, Jim agrees to an alliance wi...",Bryan Gordon,Michael Schur
4,1,Basketball,4/19/2005,8.4,3745,Michael and his staff challenge the warehouse ...,Greg Daniels,Greg Daniels


## Transform the DataFrames

In [4]:
# Create a list of columns we want to use, create a copy of the original dataframe with the chosen columns
speaker_columns = ["Season", "episode", "Title", "speaker"]
speaker_transformed = speaker_df[speaker_columns].copy()

# Rename the column headers
speaker_transformed = speaker_transformed.rename(columns={"Season": "season",
                                                          "Title": "title"})

# Create a row of ID's because we have no unique values
speaker_transformed["id"] = speaker_transformed.index+1

# Set ID as index
speaker_transformed.set_index("id", inplace=True)

# Display the transformed dataframe
speaker_transformed

,season,episode,title,speaker
id,,,,
1,1,1,Pilot,Michael
2,1,1,Pilot,Jim
3,1,1,Pilot,Michael
4,1,1,Pilot,Jim
5,1,1,Pilot,Michael
...,...,...,...,...
54623,9,24,Finale,Creed
54624,9,24,Finale,Meredith
54625,9,24,Finale,Phyllis


In [5]:
# Create a list of columns we want to use, create a copy of the original dataframe with the chosen columns
imdb_columns = ["Season", "Title", "Rating", "Num_Votes"]
imdb_transformed = imdb_df[imdb_columns].copy()

# Rename the column headers
imdb_transformed = imdb_transformed.rename(columns={"Season": "season",
                                                          "Title": "title",
                                                          "Rating": "rating",
                                                          "Num_Votes": "num_votes"})

# Create a row of ID's because we have no unique values
imdb_transformed["id"] = imdb_transformed.index+1

# Set ID as index
imdb_transformed.set_index("id", inplace=True)

# Display the transformed dataframe
imdb_transformed

,season,title,rating,num_votes
id,,,,
1,1,Pilot,7.5,4349
2,1,Diversity Day,8.3,4213
3,1,Health Care,7.8,3536
4,1,The Alliance,8.1,3428
5,1,Basketball,8.4,3745
...,...,...,...,...
372,9,Stairmageddon,8.0,1743
373,9,Paper Airplane,8.0,1748
374,9,Livin' the Dream,9.0,2439


## Create database connection

In [6]:
connection_string = "postgres:<Your Password>@localhost:5432/office_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
# Make sure the tables exist
engine.table_names()

['speaker', 'imdb']

## Load DataFrames into the Database

In [8]:
# Speaker table
speaker_transformed.to_sql(name='speaker', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "speaker_pkey"
DETAIL:  Key (id)=(1) already exists.

[SQL: INSERT INTO speaker (id, season, episode, title, speaker) VALUES (%(id)s, %(season)s, %(episode)s, %(title)s, %(speaker)s)]
[parameters: ({'id': 1, 'season': 1, 'episode': 1, 'title': 'Pilot', 'speaker': 'Michael'}, {'id': 2, 'season': 1, 'episode': 1, 'title': 'Pilot', 'speaker': 'Jim'}, {'id': 3, 'season': 1, 'episode': 1, 'title': 'Pilot', 'speaker': 'Michael'}, {'id': 4, 'season': 1, 'episode': 1, 'title': 'Pilot', 'speaker': 'Jim'}, {'id': 5, 'season': 1, 'episode': 1, 'title': 'Pilot', 'speaker': 'Michael'}, {'id': 6, 'season': 1, 'episode': 1, 'title': 'Pilot', 'speaker': 'Michael'}, {'id': 7, 'season': 1, 'episode': 1, 'title': 'Pilot', 'speaker': 'Michael'}, {'id': 8, 'season': 1, 'episode': 1, 'title': 'Pilot', 'speaker': 'Pam'}  ... displaying 10 of 54627 total bound parameter sets ...  {'id': 54626, 'season': 9, 'episod

In [ ]:
# IMDB table
imdb_transformed.to_sql(name='imdb', con=engine, if_exists='append', index=True)